In [ ]:
################################################################################
## This script contains the code to add NLS into to table.                    ##
## Mensah & Niskanen et al.                                                   ##
## Disruption of nucleolar phase separation in human genetic disease 2022     ##
## Author: Alexandre P Magalhaes                                              ##
################################################################################

In [1]:
from __future__ import print_function
import re, gzip, time, itertools, io
import sys
import pandas as pd
import csv

In [8]:
df = pd.read_csv('MasterTable_wNMD_wVariantDisease_2200511.csv',low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13002 entries, 0 to 13001
Columns: 189 entries, Uploaded_variation to ID
dtypes: bool(3), float64(150), int64(10), object(26)
memory usage: 18.5+ MB


In [5]:
#df['ID'] = df['PepID']+"_"+str(df['Amino_acids'])+"|"+str(df['Uploaded_variation'])+"|"+str(df['gene_symbol'])

In [9]:
df.tail()

,Uploaded_variation,Location,Feature,PepID,Gene,gene_symbol,Sequence,TrueIDRstart,SeqType,seqTypeExp,...,Protein_position.1,WTFullLength.1,TrueIDRstart.1,RemainingOriginalIDR,RemainingOriginalIDR%,ReplacedIDR,ReplacedIDR%,CDS_stop,NMD_check,ID
12997,NaN,NaN,ENST00000429936,ENSP00000392662,ENSG00000184216,IRAK1,MAGGPGPGEPAAPGAQHFLYEVPPWVMCRFYKVMDALEPADWCQFG...,568,WildType,WildTypeFS,...,NaN,708.0,568,NaN,NaN,NaN,NaN,2127,NMD_escaping_variant,ENSP00000392662_nan|nan|IRAK1
12998,NaN,NaN,ENST00000369835,ENSP00000358850,ENSG00000102119,EMD,MDNYADLSDTELTTLLRRYNIPHGPVVDLNSTRGDADMYDLPKKED...,99,WildType,WildTypeFS,...,NaN,219.0,99,NaN,NaN,NaN,NaN,660,NMD_escaping_variant,ENSP00000358850_nan|nan|EMD
12999,NaN,NaN,ENST00000247306,ENSP00000247306,ENSG00000126890,CTAG2,MQAEGRGTGGSTGDADGPGGPGIPDGPGGNAGGPGEAGATGGRGPR...,155,WildType,WildTypeFS,...,NaN,210.0,155,NaN,NaN,NaN,NaN,633,NMD_escaping_variant,ENSP00000247306_nan|nan|CTAG2
13000,NaN,NaN,ENST00000244174,ENSP00000244174,ENSG00000124334,IL9R,MGLGRCIWEGWTLESEALRRDMGTWLLACICICTCVCLGVSVTGEG...,413,WildType,WildTypeFS,...,NaN,521.0,413,NaN,NaN,NaN,NaN,1566,NMD_escaping_variant,ENSP00000244174_nan|nan|IL9R
13001,NaN,NaN,ENST00000383070,ENSP00000372547,ENSG00000184895,SRY,MQSYASAMLSVFNSDDYSPAVQENIPALRRSSSFLCTESCNSKYQC...,175,WildType,WildTypeFS,...,NaN,204.0,175,NaN,NaN,NaN,NaN,615,NMD_escaping_variant,ENSP00000372547_nan|nan|SRY


In [10]:
nls = pd.read_csv('Output.tsv',sep = '\t',low_memory=False)

cols = ['NLS_start', 'NLS_stop']
nls['NLS_position'] = nls[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

nls = nls.drop('NLS_algorithm', 1)
nls = nls.drop('NLS_start', 1)
nls = nls.drop('NLS_stop', 1)

nls1 = nls['NLS_score'].groupby(nls.ID).apply(list).reset_index()
nls2 = nls['NLS_position'].groupby(nls.ID).apply(list).reset_index()
nls3 = nls['NLS_sequence'].groupby(nls.ID).apply(list).reset_index()

nlsf = pd.merge(nls1, nls2, how="left", on="ID")
nls = pd.merge(nlsf, nls3, how="left", on="ID")

nls.tail()

/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/451547887.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nls = nls.drop('NLS_algorithm', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/451547887.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nls = nls.drop('NLS_start', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/451547887.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nls = nls.drop('NLS_stop', 1)


,ID,NLS_score,NLS_position,NLS_sequence
4422,ENSP00000498126_R/X|COSV63287499|AP001267.5,"[0.829, 0.994, 0.851, 0.957]","[324-332, 456-486, 723-743, 800-817]","[RKRKRKVFS, KSVEKDKSRERDREREKENKRESRKEKRKKG, K..."
4423,ENSP00000498126_T/TX|987453|AP001267.5,"[0.829, 0.994, 0.851, 0.957]","[324-332, 456-486, 723-743, 800-817]","[RKRKRKVFS, KSVEKDKSRERDREREKENKRESRKEKRKKG, K..."
4424,ENSP00000498126_T/X|827794|AP001267.5,"[0.829, 0.994, 0.851, 0.957]","[324-332, 456-486, 723-743, 800-817]","[RKRKRKVFS, KSVEKDKSRERDREREKENKRESRKEKRKKG, K..."
4425,ENSP00000498126_V/VSX|COSV63281879|AP001267.5,"[0.829, 0.994, 0.851, 0.957]","[324-332, 456-486, 723-743, 800-817]","[RKRKRKVFS, KSVEKDKSRERDREREKENKRESRKEKRKKG, K..."
4426,ENSP00000498126_nan|nan|AP001267.5,"[0.829, 0.994, 0.851, 0.957, 0.631]","[324-332, 456-486, 723-743, 800-817, 918-920]","[RKRKRKVFS, KSVEKDKSRERDREREKENKRESRKEKRKKG, K..."


In [11]:
transm = pd.read_csv('result_thhm.txt',sep = '\t',low_memory=False)
transm = transm.drop_duplicates(subset=['ID', 'Helix_Topology'], keep='first')
transm = transm.drop('len', 1)
transm = transm.drop('ExpAA', 1)
transm = transm.drop('First60', 1)
transm = transm.drop('PredHel', 1)
transm = transm['Helix_Topology'].groupby([transm.ID]).apply(list).reset_index()

transm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11208 entries, 0 to 11207
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              11208 non-null  object
 1   Helix_Topology  11208 non-null  object
dtypes: object(2)
memory usage: 175.2+ KB


/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2988621439.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  transm = transm.drop('len', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2988621439.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  transm = transm.drop('ExpAA', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2988621439.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  transm = transm.drop('First60', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2988621439.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  transm = transm.drop('PredHel', 1)


In [12]:
transm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11208 entries, 0 to 11207
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              11208 non-null  object
 1   Helix_Topology  11208 non-null  object
dtypes: object(2)
memory usage: 175.2+ KB


In [13]:
NoLs = pd.read_csv('FrameshitNoLs.txt',sep = '\t',low_memory=False)
nan_value = float("NaN")
NoLs.replace("NaN", nan_value, inplace=True)
NoLs.dropna(subset = ["NOLS"], inplace=True)

cols = ['Start', 'End']
NoLs['NoLs_position'] = NoLs[cols].apply(lambda row: '-'.join(row.values.astype(int).astype(str)), axis=1)
NoLs = NoLs.drop('Start', 1)
NoLs = NoLs.drop('End', 1)

NoLs1 = NoLs['NoLs_position'].groupby([NoLs.ID]).apply(list).reset_index()
NoLs2 = NoLs['NOLS'].groupby([NoLs.ID]).apply(list).reset_index()


NoLs = pd.merge(NoLs1, NoLs2, how="left", on="ID")

NoLs.rename(columns = {'NOLS':'NoLs_sequence'}, inplace = True)

NoLs.head()


#

/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2347402608.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  NoLs = NoLs.drop('Start', 1)
/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_72691/2347402608.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  NoLs = NoLs.drop('End', 1)


,ID,NoLs_position,NoLs_sequence
0,ENSP00000010132_-/X|COSV50542843|DYRK4,"[16-35, 402-421]","[SIKTQDPKAEEKSPKKQKVT, QSRNLKPQPRPQTLRKSNSF]"
1,ENSP00000010132_nan|nan|DYRK4,"[16-35, 401-422]","[SIKTQDPKAEEKSPKKQKVT, HQSRNLKPQPRPQTLRKSNSFF]"
2,ENSP00000011684_R/RX|COSV50614055|PLEKHG6,[754-776],[AEEPRDSRPRKLTRAQLQRMRGA]
3,ENSP00000011684_nan|nan|PLEKHG6,[754-777],[AEEPRDSRPRKLTRAQLQRMRGPH]
4,ENSP00000013807_nan|nan|ERCC1,[300-323],[KHNLRPQSFKVKKEPKTRHSGFRL]


In [14]:
df = pd.merge(df, transm, how="left", left_on="ID", right_on="ID")
df = pd.merge(df, nls, how="left", left_on="ID", right_on="ID")
df = pd.merge(df, NoLs, how="left", left_on="ID", right_on="ID")

In [15]:
df.head()

,Uploaded_variation,Location,Feature,PepID,Gene,gene_symbol,Sequence,TrueIDRstart,SeqType,seqTypeExp,...,ReplacedIDR%,CDS_stop,NMD_check,ID,Helix_Topology,NLS_score,NLS_position,NLS_sequence,NoLs_position,NoLs_sequence
0,692054,16:86511252-86511259,ENST00000262426,ENSP00000262426,ENSG00000103241,FOXF1,MSSAPEKQQPPHGGGGGGGGGGGAAMDPASSGPSKAKKTNAGIRRP...,207,Frameshift,Frameshift,...,0.860465,873,NaN,ENSP00000262426_A/X|692054|FOXF1,[nan],[0.708],[244-255],[PAAAHRRRWGHG],NaN,NaN
1,1013621,21:34792484-34792490,ENST00000399240,ENSP00000382184,ENSG00000159216,RUNX1,MASDSIFESFPSYPQCFMRECILGMNPSRDVHDASTSRRFTPPSTA...,360,Frameshift,Frameshift,...,0.310345,1773,NMD_escaping_variant,ENSP00000382184_R/X|1013621|RUNX1,[nan],"[0.974, 0.984]","[381-398, 481-534]","[RPTPARRKRREARSKPAR, PGLARLGPAGRRLRLRARGPPVRDK...","[197-216, 374-403]","[VDGPREPRRHRQKLDDQTKP, ATTPTCRRPTPARRKRREARSKP..."
2,rs1057519478,6:1611038-1611056,ENST00000645831,ENSP00000493906,ENSG00000054598,FOXC1,MQARYSVSSPNSLGVVPYLGGEQSYYRAAAAAAGGGYTAMPAPMSV...,141,Frameshift,Frameshift,...,0.856796,924,NMD_escaping_variant,ENSP00000493906_Q/X|rs1057519478|FOXC1,[nan],"[0.993, 0.996]","[169-239, 270-298]",[RRRRRFKKKDAVKDKEEKDRLHLKEPPPPGRRSRPTATRPVRSRR...,"[162-187, 217-238, 269-305]","[FENGSFLRRRRRFKKKDAVKDKEEKD, SRTSRPRTVRAPRRPSP..."
3,rs1179926739,11:17720334-17720335,ENST00000250003,ENSP00000250003,ENSG00000129152,MYOD1,MELLSPPLRDVDLTAPDGSLCSFATTDDFYDDPCFDSPDLRFFEDL...,262,Frameshift,Frameshift,...,2.310345,831,NMD_escaping_variant,ENSP00000250003_R/X|rs1179926739|MYOD1,[nan],"[0.823, 0.603, 0.657]","[186-202, 209-209, 214-221]","[PGPRRRALQRRLRRVQP, R, QRPPERRP]",[184-203],[LPPGPRRRALQRRLRRVQPA]
4,rs1603388837,18:59269381,ENST00000256852,ENSP00000256852,ENSG00000134438,RAX,MHLPGCAPAMADGSFSLAGHLLRSPGGSTSRLHSIEAILGFTKDDG...,187,Frameshift,Frameshift,...,0.779874,852,NMD_escaping_variant,ENSP00000256852_S/X|rs1603388837|RAX,[nan],"[0.616, 0.648, 0.882]","[61-65, 133-137, 245-283]","[KERDR, KKKHR, SGRRCRAGAPRRCRACRASGRRRRACLPATR...","[181-200, 251-283]","[QVWFQNRRAKWRRQEKLEVS, AGAPRRCRACRASGRRRRACLPA..."


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13002 entries, 0 to 13001
Columns: 195 entries, Uploaded_variation to NoLs_sequence
dtypes: bool(3), float64(150), int64(10), object(32)
memory usage: 19.2+ MB


In [17]:
def hidrophobic_seq(x):
    r = re.compile(r'([CAVILMFYW]..?){6,}')
    seq = str(x['Sequence'])
    span = [m.span() for m in r.finditer(seq)]
    match = [m.group() for m in r.finditer(seq)]
    return match, span
def RK_seq(x):
    r = re.compile(r'([KR]..?){6,}')
    seq = str(x['Sequence'])
    span = [m.span() for m in r.finditer(seq)]
    match = [m.group() for m in r.finditer(seq)]
    return match, span


In [18]:
df[['Hydrophobic_Match','Hydrophobic_position']] = df.apply(hidrophobic_seq, axis=1, result_type="expand")
df[['RK_Match','RK_position']] = df.apply(RK_seq, axis=1, result_type="expand")

In [20]:
df.tail()

,Uploaded_variation,Location,Feature,PepID,Gene,gene_symbol,Sequence,TrueIDRstart,SeqType,seqTypeExp,...,Helix_Topology,NLS_score,NLS_position,NLS_sequence,NoLs_position,NoLs_sequence,Hydrophobic_Match,Hydrophobic_position,RK_Match,RK_position
12997,NaN,NaN,ENST00000429936,ENSP00000392662,ENSG00000184216,IRAK1,MAGGPGPGEPAAPGAQHFLYEVPPWVMCRFYKVMDALEPADWCQFG...,568,WildType,WildTypeFS,...,[nan],NaN,NaN,NaN,[521-543],[QLACCCLHRRAKRRPPMTQENSY],"[APGAQHFLYEVPPWVMCRFYKVMDALEPADWCQF, ARVADLVHI...","[(11, 45), (103, 121), (212, 233), (341, 357),...",[],[]
12998,NaN,NaN,ENST00000369835,ENSP00000358850,ENSG00000102119,EMD,MDNYADLSDTELTTLLRRYNIPHGPVVDLNSTRGDADMYDLPKKED...,99,WildType,WildTypeFS,...,[i190-212o],NaN,NaN,NaN,NaN,NaN,[VPLWGQLLLFLVFVIVLFFIY],"[(187, 208)]",[],[]
12999,NaN,NaN,ENST00000247306,ENSP00000247306,ENSG00000126890,CTAG2,MQAEGRGTGGSTGDADGPGGPGIPDGPGGNAGGPGEAGATGGRGPR...,155,WildType,WildTypeFS,...,[nan],[0.777],[42-63],[GRGPRGAGAARASGPRGGAPRG],NaN,NaN,"[AGRMRVVGWGLGSASP, CRGVAFNVMFSAPH]","[(144, 160), (195, 209)]",[],[]
13000,NaN,NaN,ENST00000244174,ENSP00000244174,ENSG00000124334,IL9R,MGLGRCIWEGWTLESEALRRDMGTWLLACICICTCVCLGVSVTGEG...,413,WildType,WildTypeFS,...,[nan],NaN,NaN,NaN,NaN,NaN,"[MGLGRCIWEGWTLES, MGTWLLACICICTCVCL, ILRIDCHWS...","[(0, 15), (21, 38), (60, 75), (171, 187), (200...",[],[]
13001,NaN,NaN,ENST00000383070,ENSP00000372547,ENSG00000184895,SRY,MQSYASAMLSVFNSDDYSPAVQENIPALRRSSSFLCTESCNSKYQC...,175,WildType,WildTypeFS,...,[nan],[0.627],[132-134],[RRK],[119-144],[MHREKYPNYKYRPRRKAKMLPKNCSL],"[LGYQWKMLTEAEKWPFFQEAQKLQAMHR, CSLLPADPASVLCSE...","[(93, 121), (141, 159)]",[],[]


In [21]:
df.to_csv('/Users/magalhae/Desktop/MasterTable_wNMD_wVariantDisease_wRK_WHidrophobic_wNLS_wTransmembrane_220512.csv', index=False)